<a href="https://colab.research.google.com/github/ravbuilds/nvidia-nat/blob/main/climate_science_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Climate Science Chatbot**
This notebook creates and runs the NAT workflow to define agents capable of fetching, analyzing and visualizing real NOAA climate data.

###**Install Libraries**
Steps:
1. Install Nemo Agent Toolkit and LangChain dependency
2. Load API Keys - using .env file
3. Create NAT configuration
4. Run the Workflow

In [1]:
%%capture
! pip install nvidia-nat
! pip install "nvidia-nat[langchain]"

In [3]:
MODEL_BASE_URL = "https://integrate.api.nvidia.com/v1"

In [2]:
# Using Local env file
from dotenv import load_dotenv
import os

#Load enviornment vairable from .env file
load_dotenv()
# Verify the key loaded
print("API key loaded:", "Yes" if os.getenv('NVIDIA_API_KEY') else "No")

API key loaded: Yes


In [ ]:
# [RUN - IF USING GOOGLE COLAB]
# In this notebook I am loading the API keys using Google Colab Secrets
# Other options include loading a temporary .env local file to Colab runtime.

from google.colab import userdata
import os

NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')

###**Create NAT Configuration**

In [4]:
%%writefile config.yaml
llms:
  climate_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    base_url: $MODEL_BASE_URL
    api_key: $NVIDIA_API_KEY
    temperature: 0.7
    max_tokens: 2048

workflow:
  _type: chat_completion
  llm_name: climate_llm
  system_prompt: |
    You are a knowledgeable climate science assistant. You help users understand
    climate data, weather patterns, and global temperature trends. Be accurate,
    informative, and cite scientific consensus when appropriate.

Writing config.yaml


###**Test Workflow**
We will use the `NAT CLI` to run our workflow and test it.

In [5]:
!nat run \
  --config_file config.yaml \
  --input "What is the difference between weather and climate?"

2026-02-22 19:46:07 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'config.yaml'
2026-02-22 19:46:08 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function register_chat_completion by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.

Configuration Summary:
--------------------
Workflow Type: chat_completion
Number of Functions: 0
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-02-22 19:46:08 - INFO     - nat.runtime.session:298 - Shared workflow built (entry_function=None)
/Users/rsd/.pyenv/versions/3.11.7/lib/python3.11/site-packages/nat/builder/function.py:375: LangChainDeprecationWarning: Calling .text() as a method

In [6]:
!nat run \
  --config_file config.yaml \
  --input "List temperature anamolies for the top 5 warmest countries in 2025?"

2026-02-22 19:46:31 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'config.yaml'
2026-02-22 19:46:31 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function register_chat_completion by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.

Configuration Summary:
--------------------
Workflow Type: chat_completion
Number of Functions: 0
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-02-22 19:46:31 - INFO     - nat.runtime.session:298 - Shared workflow built (entry_function=None)
/Users/rsd/.pyenv/versions/3.11.7/lib/python3.11/site-packages/nat/builder/function.py:375: LangChainDeprecationWarning: Calling .text() as a method

### Deploying as an API
In production we will run `nat serve` in a terminal. Since we are running in JupyterNB, we have to run it as a subprocess.<br/>
> *nat serve will serve this agentic workflow as a self-contained api*

In [7]:
import subprocess
import time

nat_serve = subprocess.Popen(
    ["nat", "serve", "--config_file", "config.yaml"],
    stderr=subprocess.DEVNULL
)

# Run the let it print to console
time.sleep(25)
print("\n Server up and running in the background")


 Server up and running in the background


### Test the API

In [8]:
import requests
import json

response = requests.post(
    "http://localhost:8000/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    json={
        "messages": [
            {
                "role": "user",
                "content": "What causes El Nino and how does it affect global weather"
            }
        ],
        "stream": False
    }
)

if response and response.status_code == 200:
  result = response.json()
  print(result['choices'][0]['message']['content'])
else:
  print(f"Error: {response.status_code}")
  print(response.text)

2026-02-22 22:05:15 - INFO:     ::1:54285 - "POST /v1/chat/completions HTTP/1.1" 200 OK
El Niño is a complex weather phenomenon that occurs when there is an abnormal warming of the surface waters in the eastern Pacific Ocean, near the equator. This warming of the ocean water temperatures is caused by a combination of atmospheric and oceanic factors.

**Causes of El Niño:**

1. **Trade Winds:** Normally, the trade winds blow from east to west along the equator in the Pacific Ocean, pushing warm water towards Asia. During an El Niño event, these trade winds weaken or even reverse direction, allowing the warm water to flow back towards the Americas.
2. **Kelvin Waves:** Kelvin waves are oceanic waves that travel eastward along the equator. These waves can cause the warm water in the western Pacific to flow towards the eastern Pacific, contributing to the warming of the ocean waters.
3. **Atmospheric Pressure:** Changes in atmospheric pressure over the Pacific Ocean, particularly the South

### Launching a UI
Launch a UI to interact with the agent

In [10]:
# Example of using ipywidgets in Colab
from ui_manager import ui_manager
ui_manager.start()
ui_manager.show_ui_link()

🔍 Checking prerequisites...
✅ All prerequisites found
🔍 Checking NAT server on port 8000...
✅ NAT server detected on port 8000
📥 Cloning NAT UI repository...
✅ UI repository cloned
📦 Installing UI dependencies...
✅ Dependencies installed
🎨 Starting UI development server...
⏳ Waiting for service on port 3000...
✅ UI started successfully on port 3000


AttributeError: 'NoneType' object has no attribute 'format'

### Adding Intelligence with Tools
Now we will start adding multiple tools to our climate assistant. We will take plain Python function and turn them into NAT tools. Then we are going to register these tools to make them available for our agent.
Once we finish this, we will package everything.
<br/>
We will update our climate assistant to ReAct agent rather than Chat Completion.<br/>
**ReAct**: It stands for **reasoning and acting** loop. An LLM will reason and it 'll formulate one or more actions, and then call those actions.
When the LLM runs, it decides which of these tools to use.

### **NAT's Building Blocks**
**Functions**: Python functions are the foundation of NAT tools. 
To use a simple Python function as NeMo's tool, here are the steps:
1. Register it as a function within the NeMo toolkit.
2. Wrap inside an async function and async wrapper.
3. call the Python function

**Register Tool using Registration Pattern**
- Every tool starts with a *Configuration Class*. This class takes information in the YAML file
and make it available to the tool.
- Every tool has a *Input Schema*. This is a *Pydantic Class* that tells the LLM or the Agentic Workflow on how to call the tool.
- Finally register the function using Registration Decorator. This wires the tool into NeMo Agent Toolkit.